In [33]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
import json
import os
import logging
import pandas as pd
import argparse
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [48]:
args = {'model_type' : 'electra',
        'tokenizer_name' : 'google/electra-small-discriminator',
        'path_to_train' : '/content/drive/MyDrive/advNLP/twibot20/train.json',
        'path_to_val' : '/content/drive/MyDrive/advNLP/twibot20/val.json',
        'path_to_test' : '/content/drive/MyDrive/advNLP/twibot20/test.json',
        'num_train_epochs' : 10,
        'do_lower_case' : False,
        'early_stopping_consider_epochs' : False,
        'train_batch_size' : 128,
        'eval_batch_size': 128,
        'evaluate_during_training' : True,
        'learning_rate' : 4e-5, 
        'output_dir' : '/content/drive/MyDrive/advNLP/outputs/description/electra/',
        'overwrite_output_dir' : True,
        'manual_seed' : 11711,
        'use_early_stopping' : True,
        'early_stopping_metric' : 'eval_loss',
        'sliding_window' : False,
        'stride' : 0.8,
        'use_cuda' : True,
        'max_seq_length' : 128}

In [41]:
def get_dataset(path_to_data):
  with open(path_to_data) as f:
    data = json.loads(f.read())
  data = [[x['description'], 1 if x['label']=='bot' else 0] for x in data]
  df = pd.DataFrame(data)
  df.columns = ["text", "labels"]
  return df

In [42]:
def make_model(args):
  model_args = ClassificationArgs(  num_train_epochs=args['num_train_epochs'], 
                                    do_lower_case=args['do_lower_case'],
                                    early_stopping_consider_epochs=args['early_stopping_consider_epochs'],
                                    train_batch_size=args['train_batch_size'],
                                    eval_batch_size=args['eval_batch_size'],
                               	    evaluate_during_training=args['evaluate_during_training'],
                                    learning_rate=args['learning_rate'],
                                    output_dir=args['output_dir'],
                                    overwrite_output_dir=args['overwrite_output_dir'],
                                    manual_seed=args['manual_seed'],
                                    use_early_stopping=args['use_early_stopping'],
                                    early_stopping_metric=args['early_stopping_metric'],
                                    sliding_window=args['sliding_window'],
                                    stride=args['stride']
                                    
																		)
  model = ClassificationModel(args['model_type'], args['tokenizer_name'], 
                              args=model_args, use_cuda=args['use_cuda'])
  return model


def get_metrics(result):
  accuracy = (result['tp'] + result['tn']) / (result['tp'] + result['tn'] + result['fp'] + result['fn'])
  precision = result['tp'] / (result['tp'] + result['fp'])
  recall = result['tp'] / (result['tp'] + result['fn'])
  return {'acc' : accuracy, 'precision':precision, 'recall' : recall}

def train_test(args):
  logging.basicConfig(level=logging.ERROR)
  transformers_logger = logging.getLogger("transformers")
  transformers_logger.setLevel(logging.WARNING)
  
  train_data = get_dataset(args['path_to_train'])
  val_data = get_dataset(args['path_to_val'])
  test_data = get_dataset(args['path_to_test'])
  
  model = make_model(args)
  model.train_model(train_data, eval_df=val_data)
  
  result, model_outputs, wrong_predictions = model.eval_model(val_data)
  metrics = get_metrics(result)
  print("Validation results = ", metrics)
  
  result, model_outputs, wrong_predictions = model.eval_model(test_data)
  metrics = get_metrics(result)
  print("Test results = ", metrics)
  
  return model

In [49]:
model = train_test(args)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Running Evaluation:   0%|          | 0/19 [00:00<?, ?it/s]

Validation results =  {'acc': 0.6621564482029598, 'precision': 0.6752433936022253, 'recall': 0.7452033768227168}


  0%|          | 0/1183 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

Test results =  {'acc': 0.6398985629754861, 'precision': 0.6546242774566474, 'recall': 0.7078125}
